# Baseline model implication- using the pretrained YOLOv8-small

This notebook demonstrates video loading, frame extraction, these are given to the YOLOv8 and processed videos are outputted marking the cricket ball, as per detected by the model

Installing open-cv for the image and video processing

In [1]:
!pip install opencv-python

creating a dataframe containing the basic details of all the videos

In [8]:
import cv2
import pandas as pd 

df = pd.DataFrame()

for i in range(1,15):
    video_path = f"../cricket-ball-tracker/data/raw_videos/{i}.mp4"

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

    # Fetch properties
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps    = cap.get(cv2.CAP_PROP_FPS)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frames / fps if fps > 0 else 0
    new_row = pd.DataFrame({"width":width, "height":height, "fps":fps, "frame_count":frames, "duration_sec":duration}, index=[0])
    df = pd.concat([df, new_row], ignore_index=True)


### defining a function to preprocess the videos and convert them to same height, width and fps

In [ ]:
import cv2

def preprocess_video(input_path, output_path, target_width=1920, target_height=1080, target_fps=40):
    cap = cv2.VideoCapture(input_path)

    # Use target FPS
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, target_fps, (target_width, target_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize to target resolution
        frame = cv2.resize(frame, (target_width, target_height))

        out.write(frame)

    cap.release()
    out.release()
    print("Preprocessing complete! Saved to:", output_path)




### defining a function to preprocess the videos and convert them to same height, width and fps

In [13]:
for i in range(1, 15):
    preprocess_video(f"../cricket-ball-tracker/data/raw_videos/{i}.mp4", f"../cricket-ball-tracker/data/processed_videos/{i}.mp4", target_width=1920, target_height=1080, target_fps=40)

Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/1.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/2.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/3.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/4.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/5.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/6.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/7.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/8.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/9.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/10.mp4
Preprocessing complete! Saved to: ../cricket-ball-tracker/data/processed_videos/11.mp4
Preprocessing complete! Saved to: ../cricket-ball-tr

creating the dataframe of the processed videos, showing now the equal dimensions

In [ ]:
df_processed = pd.DataFrame()
for i in range(1,15):
    video_path = f"../cricket-ball-tracker/data/processed_videos/{i}.mp4"

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()

    # Fetch properties
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps    = cap.get(cv2.CAP_PROP_FPS)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frames / fps if fps > 0 else 0
    new_row = pd.DataFrame({"width":width, "height":height, "fps":fps, "frame_count":frames, "duration_sec":duration}, index=[0])
    df_processed = pd.concat([df_processed, new_row], ignore_index=True)

In [15]:
df

,width,height,fps,frame_count,duration_sec
0,1920.0,1080.0,25.000000,31.0,1.240000
1,2560.0,1600.0,57.077922,293.0,5.133333
2,2560.0,1600.0,56.956522,262.0,4.600000
3,2560.0,1600.0,56.382979,265.0,4.700000
4,2560.0,1600.0,57.411765,244.0,4.250000
5,2560.0,1440.0,38.201058,361.0,9.450000
6,2560.0,1416.0,43.404255,238.0,5.483333
7,2560.0,1440.0,42.222222,209.0,4.950000
8,2560.0,1440.0,40.942928,275.0,6.716667
9,2560.0,1440.0,42.526316,202.0,4.750000


In [16]:
df_processed 

,width,height,fps,frame_count,duration_sec
0,1920.0,1080.0,40.0,31.0,0.775
1,1920.0,1080.0,40.0,293.0,7.325
2,1920.0,1080.0,40.0,262.0,6.550
3,1920.0,1080.0,40.0,265.0,6.625
4,1920.0,1080.0,40.0,243.0,6.075
5,1920.0,1080.0,40.0,361.0,9.025
6,1920.0,1080.0,40.0,237.0,5.925
7,1920.0,1080.0,40.0,209.0,5.225
8,1920.0,1080.0,40.0,274.0,6.850
9,1920.0,1080.0,40.0,201.0,5.025


## Frame Extraction

In [ ]:
import cv2
import os

def extract_frames(video_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Save
        filename = os.path.join(output_folder, f"frame_{frame_index:06d}.jpg")
        cv2.imwrite(filename, frame)

        frame_index += 1

    cap.release()
    print("Frames extracted for {i}.mp4:", frame_index)




In [20]:
for i in range(1,15):
    extract_frames(f"../cricket-ball-tracker/data/processed_videos/{i}.mp4", f"../cricket-ball-tracker/data/frames/video_{i}")

Frames extracted: 31
Frames extracted: 293
Frames extracted: 262
Frames extracted: 265
Frames extracted: 243
Frames extracted: 361
Frames extracted: 237
Frames extracted: 209
Frames extracted: 274
Frames extracted: 201
Frames extracted: 177
Frames extracted: 138
Frames extracted: 178
Frames extracted: 196


# installing ultralytics for YOLO

In [21]:
!pip install ultralytics


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -------------- ------------------------- 2.9/8.1 MB 15.2 MB/s eta 0:00:01
   ---------------------------- ----------- 5.8/8.1 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 14.3 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 13.7 MB/s eta 0:00:00
   -----------------------------------

# defining a function to detect the ball
In the YOLOv8 we have 80 classes, out of which
sports ball = class 32 in the COCO-80 list used by YOLOv8.

In [22]:
from ultralytics import YOLO
model = YOLO("yolov8s.pt")   # pre-trained model

def detect(frame):
    results = model.predict(frame, verbose=False)[0]

    bboxes = []
    confs = []

    for box in results.boxes:
        cls = int(box.cls)
        conf = float(box.conf)

        # YOLO class 32 = sports ball (COCO)
        if cls == 32 and conf > 0.25:
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            bboxes.append([int(x1), int(y1), int(x2), int(y2)])
            confs.append(conf)

    return bboxes, confs


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\KRISHNA\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


defined a simple Tracker class

In [23]:
import numpy as np
class SimpleTracker:
    def __init__(self):
        self.next_id = 0
        self.tracks = {}  
        self.max_age = 10  

    def update(self, detections):
        assigned = {}

        if len(self.tracks) > 0 and len(detections) > 0:
            track_ids = list(self.tracks.keys())
            track_points = np.array([self.tracks[t]['centroid'] for t in track_ids])
            det_points = np.array(detections)

            # Euclidean distance
            cost = np.linalg.norm(track_points[:, None, :] - det_points[None, :, :], axis=2)

            for i in range(cost.shape[0]):
                j = np.argmin(cost[i])
                if cost[i, j] < 50:  
                    tid = track_ids[i]
                    self.tracks[tid]['centroid'] = tuple(det_points[j])
                    self.tracks[tid]['age'] = 0
                    assigned[j] = tid

        # Create new track for unassigned detections
        for i, det in enumerate(detections):
            if i not in assigned:
                tid = self.next_id
                self.next_id += 1
                self.tracks[tid] = {'centroid': det, 'age': 0}

        # Remove stale tracks
        for tid in list(self.tracks.keys()):
            self.tracks[tid]['age'] += 1
            if self.tracks[tid]['age'] > self.max_age:
                del self.tracks[tid]

        return [(tid, data['centroid']) for tid, data in self.tracks.items()]


In [24]:
import cv2
import os
import csv
import numpy as np
from collections import deque

def process_frames(
    frames_folder,
    output_video_path,
    output_csv_path,
    fps=30
):

    # Sort frames correctly
    frame_files = sorted(
        [f for f in os.listdir(frames_folder) if f.endswith(".jpg")]
    )

    if len(frame_files) == 0:
        print("ERROR: No frames found in folder:", frames_folder)
        return

    # Read first frame to get size
    first_frame = cv2.imread(os.path.join(frames_folder, frame_files[0]))
    H, W = first_frame.shape[:2]

    # Video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (W, H))

    # Tracker + trajectory
    tracker = SimpleTracker()
    trajectory = deque(maxlen=3000)

    # CSV file
    csvfile = open(output_csv_path, "w", newline="")
    writer = csv.writer(csvfile)
    writer.writerow(["frame_index", "x_centroid", "y_centroid", "visibility_flag"])

    frame_idx = 0

    # Iterate through frames
    for fname in frame_files:

        frame_path = os.path.join(frames_folder, fname)
        frame = cv2.imread(frame_path)

        # Detect
        bboxes, confs = detect(frame)

        centroids = []
        for (x1, y1, x2, y2) in bboxes:
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            centroids.append((cx, cy))

        # Update tracker
        tracks = tracker.update(centroids)

        # Write CSV
        if len(centroids) == 0:
            writer.writerow([frame_idx, 0, 0, 0])
        else:
            cx, cy = centroids[0]
            writer.writerow([frame_idx, cx, cy, 1])
            trajectory.append((cx, cy))

        # Draw detections
        for tid, (cx, cy) in tracks:
            cv2.circle(frame, (cx, cy), 8, (0, 0, 255), -1)
            cv2.putText(frame, f"{tid}", (cx + 10, cy),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # Draw trajectory
        for i in range(1, len(trajectory)):
            cv2.line(frame, trajectory[i-1], trajectory[i], (0, 255, 0), 2)

        out.write(frame)
        frame_idx += 1

    csvfile.close()
    out.release()

    print("Processing complete!")
    print("Video saved at:", output_video_path)
    print("CSV saved at:", output_csv_path)


this cell outputs out the videos

In [ ]:
for i in range(1, 15):
    process_frames(
            frames_folder=f"../cricket-ball-tracker/data/frames/video_{i}",
            output_video_path=f"../cricket-ball-tracker/data/output_videos/output_{i}.mp4",
            output_csv_path=f"../cricket-ball-tracker/data/output_csvs/output_{i}.csv",
            fps=30
        )

### Explanation
This cell performs the following operations:

(Explain here based on context.)